In [1157]:
import pandas as pd
import numpy as np
import os
import nsepy
import wget
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from datetime import datetime, timedelta

df2016 = pd.read_csv("nifty2016.csv")
df2017 = pd.read_csv("nifty2017.csv")
df2018 = pd.read_csv("nifty2018.csv")
df2019 = pd.read_csv("nifty2019.csv")
df2020 = pd.read_csv("nifty2020.csv")
df2021 = pd.read_csv("nifty2021.csv")
dfin = df2016.append(df2017)
dfin = dfin.append(df2018)
dfin = dfin.append(df2019)
dfin = dfin.append(df2020)
dfin = dfin.append(df2021)
# dfin.drop(['Shares Traded','Close','Open','Turnover (Rs. Cr)'],inplace=True,axis=1)
# dfin['Close'] = (dfin['Low'] + dfin['High']) / 2
# dfin.drop(['Low','High'],inplace=True,axis=1)
# dfin['Gap'] = dfin['Open'] - dfin['Close'].shift(1)
# dfin['Gap'] = dfin['Gap'].fillna(0)
dfin.drop(['Shares Traded','Open','Low','High','Turnover (Rs. Cr)'],inplace=True,axis=1)
dfin.head()

,Date,Close
0,26-Dec-2016,7908.25
1,27-Dec-2016,8032.85
2,28-Dec-2016,8034.85
3,29-Dec-2016,8103.60
4,30-Dec-2016,8185.80


In [1158]:
option_file = 'OptionPrice_dumpall.csv'

def load_option_prices():
    if os.path.exists(option_file):
        df = pd.read_csv(option_file)
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        df['ExpiryDay'] = pd.to_datetime(df['ExpiryDay'], format='%Y-%m-%d')
    else:
        df = pd.DataFrame(columns=['Date', 'Name', 'Type', 'StrikePrice', 'ExpiryDay','Price'])
    return df  
options_df = load_option_prices()


def get_url(dt):
    base_url = "http://www1.nseindia.com/content/historical/DERIVATIVES/" #2020/FEB/fo06FEB2020bhav.csv.zip
    yr = dt.year
    day = dt.day
    mon = dt.strftime('%b').upper()
    filename = 'fo{2:02d}{1}{0}bhav.csv.zip'.format(yr,mon,day)
    return '{0}{1}/{2}/{3}'.format(base_url,yr,mon,filename), filename

In [1159]:
load_sample = "OPTIDX_NIFTY_CE_01-Dec-2017_TO_31-Dec-20171.csv"
dfs = pd.read_csv(os.path.join("downloads",load_sample))
dfs['Expiry'] = pd.to_datetime(dfs['Expiry'], format='%d-%b-%Y')
dfs['Date'] = pd.to_datetime(dfs['Date'], format='%d-%b-%Y')
dfs["Price"]=0
dfs = dfs[['Date','Symbol',"Option Type","Strike Price","Expiry","Price","Close","Low","High","Open"]]
dfs.rename(columns = {'Symbol':'Name', 'Option Type':'Type', 'Strike Price':'StrikePrice', 'Expiry':'ExpiryDay',
                     "Close":"ClosePrice", "Low":"LowPrice", "High":"HighPrice","Open":"OpenPrice"}, inplace = True)

dfs.head()
options_df = options_df.append(dfs)
options_df['ExpiryDay'] = pd.to_datetime(options_df['ExpiryDay'], format='%Y-%m-%d')
options_df.to_csv(option_file, header=True, sep=',', index=False)

In [1160]:
options_df.tail()

,Date,Name,Type,StrikePrice,ExpiryDay,Price,ClosePrice,LowPrice,HighPrice,OpenPrice
14,2017-12-21,NIFTY,CE,10500,2017-12-28,0.0,37.10,31.65,45.15,45.05
15,2017-12-22,NIFTY,CE,10500,2017-12-28,0.0,46.35,31.25,49.85,35.70
16,2017-12-26,NIFTY,CE,10500,2017-12-28,0.0,47.90,34.85,54.70,40.70
17,2017-12-27,NIFTY,CE,10500,2017-12-28,0.0,24.70,16.50,70.00,45.80
18,2017-12-28,NIFTY,CE,10500,2017-12-28,0.0,3.80,0.05,35.00,30.00


In [1161]:
df = dfin.copy()
df['Date'] = df['Date'].apply(lambda x : datetime.strptime(x,"%d-%b-%Y"))
#df.head(20)

In [1162]:
df['Day'] = df['Date'].apply(lambda x: x.weekday())
df['Month'] = df['Date'].apply(lambda x: x.month)
df['MonthYear'] = df['Date'].apply(lambda x: str(x.year) + str(x.month))
df.head()

,Date,Close,Day,Month,MonthYear
0,2016-12-26,7908.25,0,12,201612
1,2016-12-27,8032.85,1,12,201612
2,2016-12-28,8034.85,2,12,201612
3,2016-12-29,8103.60,3,12,201612
4,2016-12-30,8185.80,4,12,201612


In [1163]:
def move_to_next_day(row):
    return_date = row['15dayback']
    if row['15dayback'].weekday() == 5:
        return_date = row['15dayback'] - timedelta(days=1)
    elif row['15dayback'].weekday() == 6:
        return_date = row['15dayback'] - timedelta(days=2)
    return return_date

def select_buy_date(row):
    dt = row['Date'].day // 2
    sel_date = row['Date'] - timedelta(days=dt)
    return sel_date

In [1164]:
df_lastthursday = df[(df['Day'] == 3) | (df['Day'] == 2)].copy()
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2017-05-31')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2018-01-31')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2018-02-28')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2018-10-31')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2019-07-31')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2020-09-30')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2021-03-31')]
df_lastthursday = df_lastthursday[(df_lastthursday['Date'] != '2021-06-30')]
df_lastthursday = df_lastthursday.groupby(['MonthYear']).tail(1).copy().reset_index(drop=True)
df_lastthursday['15dayback'] = df_lastthursday.apply(select_buy_date, axis=1)
# print(df_lastthursday.tail())
df_lastthursday['15dayback'] = df_lastthursday.apply(move_to_next_day, axis=1)
df_lastthursday['row_num'] = df_lastthursday.index
# df_lastday['15dayWeekday'] = df_lastday.apply(lambda x : x['15day'].weekday(),axis=1)
# df_lastday['row_num'] = df_lastday.index
df_lastthursday.tail(13)

,Date,Close,Day,Month,MonthYear,15dayback,row_num
48,2020-12-31,13981.75,3,12,202012,2020-12-16,48
49,2021-01-28,13817.55,3,1,20211,2021-01-14,49
50,2021-02-25,15097.35,3,2,20212,2021-02-12,50
51,2021-03-25,14324.90,3,3,20213,2021-03-12,51
52,2021-04-29,14894.90,3,4,20214,2021-04-15,52
53,2021-05-27,15337.85,3,5,20215,2021-05-14,53
54,2021-06-24,15790.45,3,6,20216,2021-06-11,54
55,2021-07-29,15778.45,3,7,20217,2021-07-15,55
56,2021-08-26,16636.90,3,8,20218,2021-08-13,56
57,2021-09-30,17618.15,3,9,20219,2021-09-15,57


In [1165]:
# change 15day back based on analysis
import calendar
months_to_change = [('2021-11','16',"U"),('2021-10','19',"U"), ('2021-09','17',"U"),('2021-08','18',"U"), ('2021-07','19',"U"),
                   ('2021-06','17',"U"),('2021-05','17',"D"),('2021-04','17',"D"),('2021-03','12',"U"),('2021-02','17',"U"),
                   ('2021-01','15',"U"),('2020-12','15',"D"),('2020-11','15',"D"),('2020-10','15',"U"),('2020-09','18',"U"),
                   ('2020-08','14',"U"),('2020-07','14',"U"),('2020-06','15',"D"),('2020-05','14',"U"),('2020-04','13',"U"),
                   ('2020-03','16',"U"),('2020-02','14',"U"),('2020-01','15',"D"),]
for mon, new_date, operation in months_to_change:
    start_date = mon + '-01'
    new_date = mon + '-' + new_date
    start_date = datetime.strptime(start_date,"%Y-%m-%d")
    end_day = calendar.monthrange(start_date.year, start_date.month)[1]
    end_date = mon + "-" + str(end_day)
    end_date = datetime.strptime(end_date,"%Y-%m-%d")
    new_date = datetime.strptime(new_date,"%Y-%m-%d")
    row_index = df_lastthursday[(df_lastthursday["Date"] >= start_date) & (df_lastthursday["Date"] <= end_date)].index[0]
    if operation == "U":
        df_lastthursday.at[row_index,'15dayback'] = new_date
    else:
        df_lastthursday = df_lastthursday.drop(row_index)
df_lastthursday = df_lastthursday[(df_lastthursday["Date"] >= '2020-01-01')]
df_lastthursday.tail(13)

,Date,Close,Day,Month,MonthYear,15dayback,row_num
44,2020-08-27,11559.25,3,8,20208,2020-08-14,44
45,2020-09-24,10805.55,3,9,20209,2020-09-18,45
46,2020-10-29,11670.80,3,10,202010,2020-10-15,46
49,2021-01-28,13817.55,3,1,20211,2021-01-15,49
50,2021-02-25,15097.35,3,2,20212,2021-02-17,50
51,2021-03-25,14324.90,3,3,20213,2021-03-12,51
54,2021-06-24,15790.45,3,6,20216,2021-06-17,54
55,2021-07-29,15778.45,3,7,20217,2021-07-19,55
56,2021-08-26,16636.90,3,8,20218,2021-08-18,56
57,2021-09-30,17618.15,3,9,20219,2021-09-17,57


In [1166]:
df_15day = df[['Date','Close']].copy()
df_mid = pd.merge(df_lastthursday, df_15day, left_on=["15dayback"], right_on=["Date"], how="left")
df_mid = df_mid[1::]
df_mid.rename(columns = {'Date_x':'Date_Buy', 'Close_x':'Close_Buy', 'Date_y':'Date_Sell', 'Close_y':'Close_Sell'}, inplace = True)

df_mid.fillna(0, inplace=True)
df_err = df_mid[(df_mid['Close_Sell'] == 0)].copy()
# df_err["15daynew"] = df_err["15day"] + timedelta(days=1)
print(len(df_err))
df_mid.tail()


0


,Date_Buy,Close_Buy,Day,Month,MonthYear,15dayback,row_num,Date_Sell,Close_Sell
13,2021-08-26,16636.90,3,8,20218,2021-08-18,56,2021-08-18,16568.85
14,2021-09-30,17618.15,3,9,20219,2021-09-17,57,2021-09-17,17585.15
15,2021-10-28,17857.25,3,10,202110,2021-10-19,58,2021-10-19,18418.75
16,2021-11-25,17536.25,3,11,202111,2021-11-16,59,2021-11-16,17999.20
17,2021-12-16,17248.40,3,12,202112,2021-12-08,60,2021-12-08,17469.75


In [1167]:
df_mid.head()

,Date_Buy,Close_Buy,Day,Month,MonthYear,15dayback,row_num,Date_Sell,Close_Sell
1,2020-03-26,8641.45,3,3,20203,2020-03-16,39,2020-03-16,9197.40
2,2020-04-30,9859.90,3,4,20204,2020-04-13,40,2020-04-13,8993.85
3,2020-05-28,9490.10,3,5,20205,2020-05-14,41,2020-05-14,9142.75
4,2020-07-30,11102.15,3,7,20207,2020-07-14,43,2020-07-14,10607.35
5,2020-08-27,11559.25,3,8,20208,2020-08-14,44,2020-08-14,11178.40


In [1168]:
for index, row in df_err.iterrows():
#     print(row['row_num'], row['15day'])
    new_date = df[(df['Date'] >= row['15dayback'])].head(1)['Date']
#     print(row['row_num'], row['15day'], new_date)
    if list(new_date):
        df_lastthursday.at[row['row_num'],'15dayback'] = list(new_date)[0]
#     else:
#         df_lastday = df_lastday.drop(index=row['row_num'])
df_lastthursday.tail()

,Date,Close,Day,Month,MonthYear,15dayback,row_num
56,2021-08-26,16636.90,3,8,20218,2021-08-18,56
57,2021-09-30,17618.15,3,9,20219,2021-09-17,57
58,2021-10-28,17857.25,3,10,202110,2021-10-19,58
59,2021-11-25,17536.25,3,11,202111,2021-11-16,59
60,2021-12-16,17248.40,3,12,202112,2021-12-08,60


In [1169]:
df_mid = pd.merge(df_lastthursday, df_15day, left_on=["15dayback"], right_on=["Date"], how="left")
df_mid.rename(columns = {'Date_x':'Date_Buy', 'Close_x':'Close_Buy', 'Date_y':'Date_Sell', 'Close_y':'Close_Sell'}, inplace = True)
df_mid = df_mid[1::]
df_mid.fillna(0, inplace=True)
df_err = df_mid[(df_mid['Close_Sell'] == 0)].copy()
if len(df_err) > 0:
    print(df_err.tail(10))
    print("Error")
else: print("No Errors")
# df_mid['Date_Buy'] = df_mid.apply(lambda x: x['Date_Buy'].date(), axis=1)
df_mid.tail()

No Errors


,Date_Buy,Close_Buy,Day,Month,MonthYear,15dayback,row_num,Date_Sell,Close_Sell
13,2021-08-26,16636.90,3,8,20218,2021-08-18,56,2021-08-18,16568.85
14,2021-09-30,17618.15,3,9,20219,2021-09-17,57,2021-09-17,17585.15
15,2021-10-28,17857.25,3,10,202110,2021-10-19,58,2021-10-19,18418.75
16,2021-11-25,17536.25,3,11,202111,2021-11-16,59,2021-11-16,17999.20
17,2021-12-16,17248.40,3,12,202112,2021-12-08,60,2021-12-08,17469.75


In [1170]:
df_mid.drop(["Day","Month","15dayback","row_num"],axis=1,inplace=True)
df_mid['Year'] = df_mid['Date_Sell'].apply(lambda x: x.year)
df_mid['BeginYear'] = df_mid['Year'].diff(periods=-1)
df_mid.loc[df_mid.index[-1], 'BeginYear']= -1
df_mid.tail()

,Date_Buy,Close_Buy,MonthYear,Date_Sell,Close_Sell,Year,BeginYear
13,2021-08-26,16636.90,20218,2021-08-18,16568.85,2021,0.0
14,2021-09-30,17618.15,20219,2021-09-17,17585.15,2021,0.0
15,2021-10-28,17857.25,202110,2021-10-19,18418.75,2021,0.0
16,2021-11-25,17536.25,202111,2021-11-16,17999.20,2021,0.0
17,2021-12-16,17248.40,202112,2021-12-08,17469.75,2021,-1.0


In [1171]:
#df_lastday.tail(20)
# spread, profit_positive, profit_negative = 0, 3300, 6700
# OTM, spread, profit_positive, profit_negative= 0, 200, 3250, 10000
# OTM, spread, profit_positive, profit_negative = 200, 100, 1650, 5000 
# OTM, spread, profit_positive, profit_negative = 100, 200, 1350, 10000 
# OTM, spread, profit_positive, profit_negative = 200, 200, 2550, 10000 
# OTM, profit_positive, profit_negative = 100, 6250, -7500
# OTM, profit_positive, profit_negative = 200, 2500, -5000
# OTM, profit_positive, profit_negative = 300, 5000, -5000
# OTM, profit_positive, profit_negative = 100, 5000, -5000
OTM, profit_positive, profit_negative = 400, 5000, -5000

In [1172]:
dfsellcall = df_mid.copy()
dfsellcall['StrikePrice'] = dfsellcall['Close_Sell'].apply(lambda x: round(x + OTM,-2))
# dfsellput['Movement'] = dfsellput['Close_Buy'] - dfsellput['StrikePrice']
# dfsellput['Profit'] = dfsellput.apply(lambda x: profit_positive if x['Movement'] > 0 else max(-profit_negative,x['Movement'] * 50),axis=1)
# dfsellput['Profit'] = dfsellput.apply(lambda x: x['Profit'] if x['Profit'] > 0 else x['Profit'] + profit_positive,axis=1)
# dfsellput['CumProfit'] = dfsellput['Profit'].cumsum()
# dfsellput['CumProfitPercentage'] = dfsellput['CumProfit'] / 115000dfsellput.head()
dfsellcall.tail()

,Date_Buy,Close_Buy,MonthYear,Date_Sell,Close_Sell,Year,BeginYear,StrikePrice
13,2021-08-26,16636.90,20218,2021-08-18,16568.85,2021,0.0,17000.0
14,2021-09-30,17618.15,20219,2021-09-17,17585.15,2021,0.0,18000.0
15,2021-10-28,17857.25,202110,2021-10-19,18418.75,2021,0.0,18800.0
16,2021-11-25,17536.25,202111,2021-11-16,17999.20,2021,0.0,18400.0
17,2021-12-16,17248.40,202112,2021-12-08,17469.75,2021,-1.0,17900.0


In [1173]:
def options_get_history(row, options_df):
    index_option = "NIFTY"
    call_strike_price = int(row['StrikePrice'])
    callopenprice, callhighprice, calllowprice, callcloseprice = 0,0,0,0
    found_price_call = options_df.loc[(options_df['Date'] == row['Date'].strftime("%Y-%m-%d")) & (options_df['Name'] == index_option) 
                                     & (options_df['Type'] == 'CE') & (options_df['StrikePrice'] == call_strike_price) 
                                     & (options_df['ExpiryDay'] == row['ExpiryDay'].strftime("%Y-%m-%d"))]
    if len(found_price_call) == 0: 
        
        #http://www1.nseindia.com/content/historical/DERIVATIVES/2020/FEB/fo06FEB2020bhav.csv.zip
        url, filename = get_url(row['Date'])
        print('Fetching Price from NSE dumps {0}_{1}_CE on {2}. FileName: {3}'.format(row['ExpiryDay'], call_strike_price, row['Date'].strftime("%Y-%m-%d"),url))
        op_file = os.path.join('downloads',filename)
        if not os.path.exists(op_file):
            wget.download(url, out='./downloads')
            print()
        df = pd.read_csv(op_file)
        df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'], format='%d-%b-%Y')
        df = df[df['INSTRUMENT'] == 'OPTIDX']
        df = df[df['SYMBOL'] == index_option]
        df = df[df['STRIKE_PR'] == call_strike_price]
        df = df[df['EXPIRY_DT'] == row['ExpiryDay'].strftime("%Y-%m-%d")]
        callopenprice = df[df['OPTION_TYP'] == 'CE']['OPEN'].values[0]
        callhighprice = df[df['OPTION_TYP'] == 'CE']['HIGH'].values[0]
        calllowprice = df[df['OPTION_TYP'] == 'CE']['LOW'].values[0]
        callcloseprice = df[df['OPTION_TYP'] == 'CE']['CLOSE'].values[0]
        temp_dict = {}
        temp_dict[row['Date']] = {'Name'          : index_option, 
                                    'Type'        : 'CE', 
                                    'StrikePrice' : call_strike_price,
                                    'ExpiryDay'   : row['ExpiryDay'],
                                    'ClosePrice'  : callcloseprice,
                                    'LowPrice'    : calllowprice,
                                    'HighPrice'   : callhighprice,
                                    'OpenPrice'   : callopenprice,
                                    }
        temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
        temp_df['Date'] = temp_df.index
        temp_df.reset_index(inplace=True,drop=True)
        options_df = options_df.append(temp_df, ignore_index = True,sort=False)
        
        temp_df = None 
    else:
        callopenprice = found_price_call['OpenPrice'].values[0]
        callhighprice = found_price_call['HighPrice'].values[0]
        calllowprice = found_price_call['LowPrice'].values[0]
        callcloseprice = found_price_call['ClosePrice'].values[0]
    return callopenprice, callhighprice, calllowprice, callcloseprice, options_df

In [1174]:
dfsellcall.tail()

,Date_Buy,Close_Buy,MonthYear,Date_Sell,Close_Sell,Year,BeginYear,StrikePrice
13,2021-08-26,16636.90,20218,2021-08-18,16568.85,2021,0.0,17000.0
14,2021-09-30,17618.15,20219,2021-09-17,17585.15,2021,0.0,18000.0
15,2021-10-28,17857.25,202110,2021-10-19,18418.75,2021,0.0,18800.0
16,2021-11-25,17536.25,202111,2021-11-16,17999.20,2021,0.0,18400.0
17,2021-12-16,17248.40,202112,2021-12-08,17469.75,2021,-1.0,17900.0


In [1175]:
transcallsellprice, transcallhigh, transcalllow, transcallbuyprice, transcallhighdate, transcalllowdate = [],[],[],[],[],[]
for i, r in dfsellcall.iterrows():
    expiry = list(sorted(get_expiry_date(year=r['Year'], month=r['Date_Buy'].month, stock=True)))[-1]
    dfw = df[(df["Date"] >= r['Date_Sell']) & (df["Date"] < r['Date_Buy'])][['Date','Close']]
    row = {
           "Date": r['Date_Sell'], 
           "Name": "NIFTY",
           "StrikePrice": r['StrikePrice'],
           "ExpiryDay": expiry
          }
    callopen, callhigh, calllow, callclose, call_date = [], [], [], [], []
    for ii, ir in dfw.iterrows():
        row['Date'] = ir['Date']
        callopenprice, callhighprice, calllowprice, callcloseprice, options_df = options_get_history(row, options_df)
        callopen.append(callopenprice)
        callhigh.append(callhighprice)
        calllow.append(calllowprice)
        callclose.append(callcloseprice)
        call_date.append(row['Date'])
#         print(callhighprice, calllowprice, callcloseprice)

    transcallsellprice.append(callclose[0])
    transcallbuyprice.append(callclose[-1])
    
    maxi = [(i,j.strftime("%Y-%m-%d")) for i,j in zip(callhigh[1:],call_date[1:]) if i >= callclose[0] * 1.5]
    if maxi:
        max_call, max_date = maxi[0]
    else:
        max_call, max_date= max(callhigh[1:]),0
    transcallhigh.append(max_call)
    transcallhighdate.append(max_date)
    lowe = [(i,j.strftime("%Y-%m-%d")) for i,j in zip(calllow[1:],call_date[1:]) if i <= callclose[0] * 0.5]
    if lowe:
        low_call, low_date = lowe[-1]
    else:
        low_call, low_date= max(calllow[1:]), 0
    transcalllow.append(low_call)
    transcalllowdate.append(low_date)

#     print()
#     print(transcallsellprice, transcallhigh, transcalllow, transcallbuyprice)
#     break
dfsellcall['transcallsellprice'] = transcallsellprice
dfsellcall['transcallhigh'] = transcallhigh
dfsellcall['transcallhighdate'] = transcallhighdate
dfsellcall['transcalllow'] = transcalllow
dfsellcall['transcalllowdate'] = transcalllowdate
dfsellcall['transcallbuyprice'] = transcallbuyprice
dfsellcall.head()

Fetching Price from NSE dumps 2020-03-26_9600_CE on 2020-03-16. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/MAR/fo16MAR2020bhav.csv.zip
Fetching Price from NSE dumps 2020-03-26_9600_CE on 2020-03-17. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/MAR/fo17MAR2020bhav.csv.zip
Fetching Price from NSE dumps 2020-03-26_9600_CE on 2020-03-18. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/MAR/fo18MAR2020bhav.csv.zip
Fetching Price from NSE dumps 2020-03-26_9600_CE on 2020-03-19. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/MAR/fo19MAR2020bhav.csv.zip
Fetching Price from NSE dumps 2020-03-26_9600_CE on 2020-03-20. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/MAR/fo20MAR2020bhav.csv.zip
Fetching Price from NSE dumps 2020-03-26_9600_CE on 2020-03-23. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/MAR/fo23MAR2020bhav.csv.zip
Fetching Price from NS

Fetching Price from NSE dumps 2020-09-24_11900_CE on 2020-09-22. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/SEP/fo22SEP2020bhav.csv.zip
Fetching Price from NSE dumps 2020-09-24_11900_CE on 2020-09-23. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/SEP/fo23SEP2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_12100_CE on 2020-10-15. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo15OCT2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_12100_CE on 2020-10-16. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo16OCT2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_12100_CE on 2020-10-19. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo19OCT2020bhav.csv.zip
Fetching Price from NSE dumps 2020-10-29_12100_CE on 2020-10-20. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2020/OCT/fo20OCT2020bhav.csv.zip
Fetching Price f

Fetching Price from NSE dumps 2021-08-26_17000_CE on 2021-08-25. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/AUG/fo25AUG2021bhav.csv.zip
Fetching Price from NSE dumps 2021-09-30_18000_CE on 2021-09-17. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/SEP/fo17SEP2021bhav.csv.zip
Fetching Price from NSE dumps 2021-09-30_18000_CE on 2021-09-20. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/SEP/fo20SEP2021bhav.csv.zip
Fetching Price from NSE dumps 2021-09-30_18000_CE on 2021-09-21. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/SEP/fo21SEP2021bhav.csv.zip
Fetching Price from NSE dumps 2021-09-30_18000_CE on 2021-09-22. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/SEP/fo22SEP2021bhav.csv.zip
Fetching Price from NSE dumps 2021-09-30_18000_CE on 2021-09-23. FileName: http://www1.nseindia.com/content/historical/DERIVATIVES/2021/SEP/fo23SEP2021bhav.csv.zip
Fetching Price f

,Date_Buy,Close_Buy,MonthYear,Date_Sell,Close_Sell,Year,BeginYear,StrikePrice,transcallsellprice,transcallhigh,transcallhighdate,transcalllow,transcalllowdate,transcallbuyprice
1,2020-03-26,8641.45,20203,2020-03-16,9197.40,2020,0.0,9600.0,314.15,362.0,0,2.10,2020-03-25,4.20
2,2020-04-30,9859.90,20204,2020-04-13,8993.85,2020,0.0,9400.0,200.45,284.6,0,78.40,2020-04-29,167.55
3,2020-05-28,9490.10,20205,2020-05-14,9142.75,2020,0.0,9500.0,113.25,115.7,0,0.85,2020-05-27,6.45
4,2020-07-30,11102.15,20207,2020-07-14,10607.35,2020,0.0,11000.0,66.40,120.8,2020-07-15,190.20,0,229.30
5,2020-08-27,11559.25,20208,2020-08-14,11178.40,2020,0.0,11600.0,27.85,46.9,2020-08-18,6.15,2020-08-26,18.50


In [1176]:
dfsellcall['RiskPrice'] = dfsellcall['transcallsellprice'] * 1.5
dfsellcall['StopLossHit'] = dfsellcall.apply(lambda x: "Yes" if x['transcallhigh'] - x['RiskPrice'] > 0 else "No", axis=1)
dfsellcall['StopLossHitinDays'] = dfsellcall.apply(lambda x: (datetime.strptime(x["transcallhighdate"],"%Y-%m-%d") - x['Date_Sell']).days if x['StopLossHit'] == "Yes" else "0", axis=1) 
dfsellcall['Profit'] = dfsellcall.apply(lambda x: (x['transcallsellprice'] - x['transcallbuyprice']) * 50 if x['StopLossHit'] == 'No' else (x['transcallsellprice'] - x['RiskPrice']) * 50 , axis=1)
dfsellcall['SellMargin'] = dfsellcall['Close_Sell'] * 6
dfsellcall['ProfitPercentage'] = dfsellcall['Profit'] / dfsellcall['SellMargin']
dfsellcall['CumProfit'] = dfsellcall['Profit'].cumsum()
dfsellcall['CumSellMargin'] = dfsellcall['SellMargin'].cumsum()
dfsellcall['row_num'] = dfsellcall.index + 1
dfsellcall['CumSellMargin'] = dfsellcall['CumSellMargin']  / dfsellcall['row_num']
dfsellcall['CumProfitPercentage'] = dfsellcall['CumProfit'] / dfsellcall['CumSellMargin']

dfsellcall.head()

,Date_Buy,Close_Buy,MonthYear,Date_Sell,Close_Sell,Year,BeginYear,StrikePrice,transcallsellprice,transcallhigh,...,RiskPrice,StopLossHit,StopLossHitinDays,Profit,SellMargin,ProfitPercentage,CumProfit,CumSellMargin,row_num,CumProfitPercentage
1,2020-03-26,8641.45,20203,2020-03-16,9197.40,2020,0.0,9600.0,314.15,362.0,...,471.225,No,0,15497.50,55184.4,0.280831,15497.50,27592.20,2,0.561662
2,2020-04-30,9859.90,20204,2020-04-13,8993.85,2020,0.0,9400.0,200.45,284.6,...,300.675,No,0,1645.00,53963.1,0.030484,17142.50,36382.50,3,0.471174
3,2020-05-28,9490.10,20205,2020-05-14,9142.75,2020,0.0,9500.0,113.25,115.7,...,169.875,No,0,5340.00,54856.5,0.097345,22482.50,41001.00,4,0.548340
4,2020-07-30,11102.15,20207,2020-07-14,10607.35,2020,0.0,11000.0,66.40,120.8,...,99.600,Yes,1,-1660.00,63644.1,-0.026083,20822.50,45529.62,5,0.457340
5,2020-08-27,11559.25,20208,2020-08-14,11178.40,2020,0.0,11600.0,27.85,46.9,...,41.775,Yes,4,-696.25,67070.4,-0.010381,20126.25,49119.75,6,0.409738


In [1177]:
# dfsellput.drop(['Gap','Gap_Open','Gap_Date','Open','Low','MonthYear','Year'],inplace=True,axis=1)
dfsellcall.to_csv("nifty_sellcall_optiondata_OTM" + str(OTM) +".csv")

In [1178]:
# options_df['ExpiryDay'] = options_df['ExpiryDay'].strftime("%Y-%m-%d")
options_df.drop_duplicates()
options_df.to_csv(option_file, header=True, sep=',', index=False)